Since in this competition the leaderboard does not really matter, as all test data is included in the training set, I was simply curious to see what the maximum possible score of the metric could be if we had perfect knowledge of the "future" market behavior, and to better understand how the evaluation metric works.

(And it was also fun to get to the first position on the leaderboard at least once in my life, even if only for a short while =)

Update: Actually, with a fixed strategy (which also knows future prices), the best I’ve found is to skip on the loss-making days and, on the profitable ones, return a stake of about 0.1 (I clarified this in the new version).  

I think the strategy can be improved into something more flexible, but I haven’t figured out how to do that yet.  

If this problem could be solved, then for achieving a perfect result only one tiny detail would remain — fully predicting the behavior of the market 😂

In [ ]:
import os

import pandas as pd
import polars as pl
from pathlib import Path

import kaggle_evaluation.default_inference_server


DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}


def predict(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)    
    return 0.09 if t > 0 else 0.0


inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))